In [1]:
import kratos
from agno.agent import Agent
from agno.models.lmstudio import LMStudio
from agno.tools.youtube import YouTubeTools
from agno.tools.yfinance import YFinanceTools

# Creating a few agents using the Agno framework for building agents
# All the llm models here use LMStudio as the provider and connect to local API server
# I'm using qwen/qwen3-4b model for both agents, but you can use any other model that you have access to.

finance_agent = Agent(
    model=LMStudio(
        id="qwen/qwen3-4b",
        base_url="http://host.docker.internal:1234/v1",
    ),
    name="KratosFinance",
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True)],
    instructions="You are Kratos Finance Agent, specialized in stock market analysis and financial data. Use YFinance to get current stock prices, analyst recommendations, and company information. When users ask about stocks, always provide current prices and relevant financial metrics. Be precise with numbers and explain what the data means for investment decisions.",
)


youtube_agent = Agent(
    model=LMStudio(
        id="qwen/qwen3-4b",
        base_url="http://host.docker.internal:1234/v1",
    ),
    name="KratosYouTube",
    tools=[YouTubeTools()],
    instructions="You are Kratos YouTube Agent, specialized in analyzing YouTube videos and transcripts. Use YouTube tools to extract video information, transcripts, and analyze content. When users mention YouTube videos, provide detailed analysis of the content, key points, and insights from the transcript. Be thorough in your analysis and highlight important information.",
)


In [2]:
# Submit these agents so that they are ready to be used at a later point - none of
# these models are actually loaded in memory right now and thus no resources are wasted.

print(kratos.submit(finance_agent, name="agent-finance", dependencies=["yfinance"]))
print(kratos.submit(youtube_agent, name="agent-youtube", dependencies=["youtube-transcript-api"]))

agent-finance
agent-youtube


In [3]:
# The name parameter we used in submit() acts as an identifier as to which agent do you want to invoke.

kratos.invoke(name="agent-finance", task="How has the trend been for NVIDIA stock?")

<think>
Okay, the user is asking about the trend for NVIDIA's stock. Let me figure out how to approach this.
First, I need to determine what information is required. The user wants to know the trend, which likely involves recent performance, price movements, and maybe analyst opinions. 
I should start by getting the current stock price for NVIDIA. The function get_current_stock_price is needed here. The symbol for NVIDIA is NVDA. So I'll call that function to get the latest price.
Next, to understand the trend, I might need historical data, but since the available functions don't include historical data retrieval, I can only provide the current price. However, the user might also want to know analyst recommendations. So I should check the analyst opinions using get_analyst_recommendations. 
Additionally, company information could provide context about NVIDIA's performance and future outlook. So using get_company_info might help explain the current situation. 
Wait, but the user specifi

In [6]:
kratos.invoke(name="agent-youtube", task="Get me a summary of this video: https://www.youtube.com/watch?v=dQw4w9WgXcQ")

<think>
Okay, the user wants a summary of the YouTube video with the URL https://www.youtube.com/watch?v=dQw4w9WgXcQ. Let me figure out how to approach this.
First, I need to use the tools provided. The available functions are get_youtube_video_captions, get_youtube_video_data, and get_video_timestamps. The user is asking for a summary, which likely involves the video's content. So, I should start by getting the captions since they contain the transcript text. 
But wait, maybe I should check the video data first to get details like title and author, which could be part of the summary. However, the main content for a summary would probably come from the captions. So I'll need to call get_youtube_video_captions with the provided URL. Once I have the captions, I can analyze them to extract key points and create a summary. 
Alternatively, maybe get_video_timestamps could help in structuring the timeline of the video, but the user hasn't specified a particular aspect beyond the summary. The

In [ ]:
from concurrent.futures import ThreadPoolExecutor, wait

print(f"\n🚀 Starting 3 concurrent invocations where each instance of an agent, will have it's own model loaded in memory")

tasks = [
    "Stock price of NVIDIA",
    "Stock price of Apple",
    "Stock price of Tesla",
]

with ThreadPoolExecutor(max_workers=3) as executor:
    futures = [executor.submit(kratos.invoke, 'agent-finance', task, False) for task in tasks]

    print(f"\n🔄 Waiting for all agents to complete...")
    wait(futures)

    print(f"\n🔄 All agents have completed their tasks")

print("\n\nQ&A:")


🚀 Starting 3 concurrent invocations where each instance of an agent, will have it's own model loaded in memory

🔄 Waiting for all agents to complete...

🔄 All agents have completed their tasks
